In [ ]:
import torch
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# load data and predictions
x = # input data
y = # true locations
y_hat = # predictions

n, m = [*y.shape]

In [ ]:
# define d to be either interocular distance or bounding box diagonal length
d = 

In [ ]:
""" Accuracy Metrics: Normalized Mean Error (NME) and CED/AUC/FR
y, y_hat have dimensions of num_samples x (2*num_features)
d is the interocular distance
"""
# returns the NME of each image in the dataset
def nme(y_hat, y, m, d):
    return torch.square(torch.norm(y_hat-y, dim=1))/ d / m

# returns the average NME across all images
def total_nme(nme):
    return torch.mean(NME)

# outputs the CED
def ced(nme):
    x = torch.sort(NME)
    y = np.arange(n) / float(n)
    plt.xlabel('x-axis')
    plt.ylabel('y-axis')
  
    plt.title('Plot of NME vs Fraction of Samples')
    plt.plot(x, y, marker='o')
    plt.show

# returns the percentage of samples with nme=0 to alpha
def auc(nme, alpha):
    x = torch.where(NME < alpha, 1, 0)
    return torch.sum(failure) / n
    
# returns the failure rate, where failure is defined w.r.t a threshold, alpha (percent of samples with nme>alpha)
def fr(nme, alpha):
    return 1-auc(nme, alpha)

In [ ]:
nme = nme(y_hat, y, m, d)
total_nme = total_nme(nme)

In [ ]:
ced = ced(nme)

In [ ]:
"""Bias Metrics: Majority/Minority Difference (MMD), Improvement Over the Standard Model (IOSM), KL-divergence
g = sensitive attribute group (as column number)
maj = value of minority group
nme = num_samples dimension tensor
"""

def mmd(g, maj, nme, data):
    acc = 1-nme
    majority = acc[tensor.where(data[:, g] == maj)]
    minority = acc[tensor.where(data[:, g] != maj)]
    difference = torch.mean(majority) - torch.mean(minority)
    return difference

# def iosm(g, nme, )

# def KL divergence(g, maj, nme, data):
